In [ ]:
%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torchvision.utils import save_image
import torchvision.utils as vutils
import numpy as np
import pywt
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import sys
import gc
sys.path.append('../wtvae/src/')
from vae_models import WT, wt, IWT, iwt, IWTAE_128_Mask_2, IWTAE_512_Mask_2
from utils.utils import zero_patches, zero_mask, set_seed, save_plot, create_inv_filters, create_filters, zero_pad

# Set random seem for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [ ]:
# Root directory for dataset
dataroot = '/hdd_e/han/data/celeba_512/'

# Number of workers for dataloader
workers = 4

# Batch size during training
batch_size = 128

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 512

# Number of channels in the training images. For color images this is 3
nc = 3

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 128

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 300

# Learning rate for optimizers
lr_D = 0.0002
lr_G = 0.0002
lr_mn = 0.0001

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

In [ ]:
# We can use an image folder dataset the way we have it setup.
# Create the dataset
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(512),
                               transforms.CenterCrop(512),
                               transforms.ToTensor(),
#                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers, drop_last=True)

# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# Plot some training images
real_batch = next(iter(dataloader))

# print(np.min(real_batch[0][0].numpy()))

plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:24], padding=2, normalize=False).cpu(),(1,2,0)))


In [ ]:
def wt(vimg, filters, levels=1):
    bs = vimg.shape[0]
    h = vimg.size(2)
    w = vimg.size(3)
    vimg = vimg.reshape(-1, 1, h, w)
    padded = torch.nn.functional.pad(vimg,(2,2,2,2))
    res = torch.nn.functional.conv2d(padded, Variable(filters[:,None]),stride=2)
    if levels>1:
        res[:,:1] = wt(res[:,:1], filters, levels-1)
        res[:,:1,32:,:] = res[:,:1,32:,:]*1.
        res[:,:1,:,32:] = res[:,:1,:,32:]*1.
        res[:,1:] = res[:,1:]*1.
    res = res.view(-1,2,h//2,w//2).transpose(1,2).contiguous().view(-1,1,h,w)
    return res.reshape(bs, -1, h, w)

def create_filters(device, wt_fn='bior2.2'):
    w = pywt.Wavelet(wt_fn)
    dec_hi = torch.Tensor(w.dec_hi[::-1]).to(device)
    dec_lo = torch.Tensor(w.dec_lo[::-1]).to(device)
    filters = torch.stack([dec_lo.unsqueeze(0)*dec_lo.unsqueeze(1),
                           dec_lo.unsqueeze(0)*dec_hi.unsqueeze(1),
                           dec_hi.unsqueeze(0)*dec_lo.unsqueeze(1),
                           dec_hi.unsqueeze(0)*dec_hi.unsqueeze(1)], dim=0)
    return filters

def iwt(vres, inv_filters, levels=1):
    bs = vres.shape[0]
    h = vres.size(2)
    w = vres.size(3)
    vres = vres.reshape(-1, 1, h, w)
    res = vres.contiguous().view(-1, h//2, 2, w//2).transpose(1, 2).contiguous().view(-1, 4, h//2, w//2).clone()
    if levels > 1:
        res[:,:1] = iwt(res[:,:1], inv_filters, levels=levels-1)
    res = torch.nn.functional.conv_transpose2d(res, Variable(inv_filters[:,None]),stride=2)
    res = res[:,:,2:-2,2:-2] #removing padding

    return res.reshape(bs, -1, h, w)

filters = create_filters(device='cuda:0')
inv_filters = create_filters(device='cuda:0')

cur_max = float('-inf')
cur_min = float('inf')
for i, data in enumerate(dataloader):
    data_512 = data[0].to('cuda:0')
    data_wt = wt(data_512, filters=filters, levels=4)[:, :, :32, :32]
    cur_max = max(cur_max, torch.max(data_wt))
    cur_min = min(cur_min, torch.min(data_wt))
print(cur_max)
print(cur_min)

In [ ]:
shift = -1*cur_min
scale = shift+cur_max

In [ ]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)
        
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [ ]:
# Variational Autoencoder Code
'''Sub class the nn module'''
class AE(nn.Module):
    def __init__(self, ngpu):
        super(AE, self).__init__() #not a python thing
        self.ngpu = ngpu
        
        self.encoder = nn.Sequential(
            nn.Conv2d(nc, ndf//2, 4, 2, 1, bias=False),
            nn.ReLU(True),
            # input is (nc) x 64 x 64
            nn.Conv2d(ndf//2, ndf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf),
            nn.ReLU(True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.ReLU(True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.ReLU(True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.ReLU(True),
            # state size. (ndf*8) x 4 x 4
            Flatten()
        )
        
        self.fc1 = nn.Linear(ndf*8*4*4, nz)
        self.fc2 = nn.Linear(ndf*8*4*4, nz)
        self.fc3 = nn.Linear(nz, ndf*8*4*4)
        
        self.decoder = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( ndf*8*4*4, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, ngf//2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf//2),
            nn.ReLU(True),
            
            nn.ConvTranspose2d( ngf//2, nc, 4, 2, 1, bias=False),
            nn.Sigmoid()
            # state size. (nc) x 64 x 64
        )
        
    def reparameterize(self, mu, logvar):
        eps = torch.randn(*mu.size(),device=device)
        std = logvar.mul(0.5).exp_()
        # return torch.normal(mu, std)
        z = mu + (std*eps)
        return z
    
    def bottleneck(self, h):
        mu, logvar = self.fc1(h), self.fc2(h)
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar

    def encode(self, x):
        h = self.encoder(x)
        z, mu, logvar = self.bottleneck(h)
        return z, mu, logvar

    def decode(self, z):
        z = self.fc3(z)
        z = z.unsqueeze(-1).unsqueeze(-1)
        x = self.decoder(z)
        return x

    def forward(self, x):
        z, mu, logvar = self.encode(x)
        x = self.decode(z)
        return x, mu, logvar

`Input: (N,Cin,Hin,Win)
Output: (N,Cout,Hout,Wout) where Hout=(Hin−1)∗stride[0]−2∗padding[0]+kernel_size[0]+output_padding[0]  Wout=(Win−1)∗stride[1]−2∗padding[1]+kernel_size[1]+output_padding[1]`

In [ ]:
# Decide which device we want to run on
device_2 = torch.device("cuda:1" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
# Create a copy of filters on the second device
filters_d2 = create_filters(device=device_2)

# Generator Code
'''Sub class the nn module'''
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__() #not a python thing
        self.ngpu = ngpu
        self.encoder = nn.Sequential(
            nn.Conv2d(nc, ndf//2, 4, 2, 1, bias=False),
            nn.ReLU(True),
            # input is (nc) x 64 x 64
            nn.Conv2d(ndf//2, ndf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf),
            nn.ReLU(True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.ReLU(True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.ReLU(True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.ReLU(True),
            # state size. (ndf*8) x 4 x 4
            Flatten()
        )
        
        self.decoder = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( ndf*8*4*4, ngf * 8, 4, 1, 0, bias=False),
            nn.InstanceNorm2d(ngf * 8,affine=True),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(ngf * 4,affine=True),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(ngf * 2,affine=True),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(ngf,affine=True),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, ngf//2, 4, 2, 1, bias=False),
            nn.InstanceNorm2d(ngf//2,affine=True),
            nn.ReLU(True),
            
            nn.ConvTranspose2d( ngf//2, nc, 4, 2, 1, bias=False),
            nn.Sigmoid()
            # state size. (nc) x 64 x 64
        )

    def forward(self, input):
#         eps = torch.randn(b_size, 3, 64, 64, device=device)
        enc = self.encoder(input)
        enc = enc.unsqueeze(-1).unsqueeze(-1)
        return self.decoder(enc)

In [ ]:
def plot():
    with torch.no_grad():
        netAE.eval()
        data = next(iter(dataloader))
        img = data[0][:32].to(device)
        img = wt(img, filters=filters, levels=2)[:, :, :128, :128]
        img = (img+shift)/scale
        b_size = img.size(0)
        grid,_,_ = netAE(img)
        grid = grid.detach().cpu()
        plt.figure(figsize=(8,8))
        plt.axis("off")
        plt.title("AE Reconstruction")
        plt.imshow(np.transpose(vutils.make_grid(grid, padding=2, normalize=False).cpu(),(1,2,0)))
        
        netG.eval()
        grid = netG(grid.to(device_2)).detach().cpu()
        plt.figure(figsize=(8,8))
        plt.axis("off")
        plt.title("GAN Reconstruction")
        plt.imshow(np.transpose(vutils.make_grid(grid, padding=2, normalize=False).cpu(),(1,2,0)))
        plt.show()

In [ ]:
DEVICE0 = 'cuda:0'
DEVICE1 = 'cuda:1'

checkpoint_dsvae_path = '/hdd_e/han/wtvae_results/models/dsvae_wt/DSVAE_WT.pt'
# Load AE model for DSVAE
netAE = AE(0).to(DEVICE0)
checkpoint_dsvae = torch.load(checkpoint_dsvae_path, map_location=DEVICE0)
netAE.load_state_dict(checkpoint_dsvae['netAE_state_dict'])
netAE.eval()

# Load DSVAE
dsvae = Generator(0).to(DEVICE0)
dsvae.load_state_dict(checkpoint_dsvae['netG_state_dict'])
dsvae.eval()

In [ ]:
data = next(iter(dataloader))
img = data[0][:32].to(device)
img = wt(img, filters=filters, levels=2)[:, :, :128, :128]
img = (img+shift)/scale
grid,_,_ = netAE(img)
grid_dsvae = dsvae(grid).detach()
plt.imshow(np.transpose(vutils.make_grid(grid_dsvae, padding=2, normalize=False).cpu(),(1,2,0)))
plt.show()
grid_dsvae_denorm = grid_dsvae * scale - shift
plt.imshow(np.transpose(vutils.make_grid(grid_dsvae_denorm, padding=2, normalize=False).cpu(),(1,2,0)))

In [ ]:
# Load saved 128 and 512 mask networks
# checkpoint_128_path = '/hdd_e/han/wtvae_results/models/iwtae128_noimgloss_lr1e-4_nonorm_2iwt_full/iwtvae_epoch300.pth'

# iwt_model_128 = IWTAE_128_Mask_2(z_dim=100, num_iwt=2)
# iwt_model_128.set_filters(inv_filters)
# iwt_model_128.set_device(DEVICE1)
# iwt_model_128 = iwt_model_128.to(DEVICE1)

# checkpoint_128 = torch.load(checkpoint_128_path, map_location=DEVICE1)
# iwt_model_128.load_state_dict(checkpoint_128['model_state_dict'])

checkpoint_512_path = '/hdd_e/han/wtvae_results/models/iwtae512_overfit_noimgloss_lr1e-4_nonorm_full_rerun/iwtvae_epoch272.pth'
iwt_model_512 = IWTAE_512_Mask_2(z_dim=100, num_iwt=3)
iwt_model_512.set_device(DEVICE1)
iwt_model_512 = iwt_model_512.to(DEVICE1)

checkpoint_512 = torch.load(checkpoint_512_path, map_location=DEVICE1)
iwt_model_512.load_state_dict(checkpoint_512['model_state_dict'])

inv_filters = create_inv_filters(device=DEVICE0)
iwt_fn = IWT(iwt=iwt, num_iwt=2)
iwt_fn.set_filters(inv_filters)

In [ ]:
def denormalize(data, shift, scale):
    return data*scale - shift
    
def normalize(data, shift, scale):
    return (data + shift) / scale

In [ ]:
# optimizer_mn_128 = optim.Adam(iwt_model_128.parameters(), lr=lr_mn)
optimizer_mn_512 = optim.Adam(iwt_model_512.parameters(), lr=lr_mn)

In [ ]:
iwt_model_128.train()

# Finetuning 128 mask network
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):
        optimizer_mn_128.zero_grad()
        
        real_cpu = data[0].to(DEVICE0)
        Y = wt(real_cpu, filters, levels=4)
        Y_128 = wt(real_cpu, filters, levels=2)[:, :, :128, :128]
        
        # Get real mask at 128 x 128 (2 WT on that so each patch is 32 x 32)
        real_mask = Y[:, :, :128, :128]
        real_mask = zero_mask(real_mask, 2, 1)
        real_mask = iwt(real_mask, inv_filters, levels=2)
        
        # Get reconstruction from AE & DSVAE
        Y_128_norm = normalize(Y_128, shift, scale)
        Y_128_norm_hat, _, _ = netAE(Y_128_norm)
        Y_128_norm_hat = dsvae(Y_128_norm_hat)
        Y_128_hat = denormalize(Y_128_norm_hat, shift, scale)
        
        # Apply WT twice to get 32 x 32 low freq patch and pad with zeros to create 128 x 128
        Y_32 = wt(Y_128_hat, filters, levels=2)[:, :, :32, :32]
        Y_32_padded = zero_pad(Y_32, 128, device=DEVICE0)
        Y_32_low = iwt(Y_32_padded, inv_filters, levels=2)
        
        # Run through 128 mask network and backpropagate
        recon_mask, mu, var = iwt_model_128(Y_32_low.to(DEVICE1))
        loss, _, _ = iwt_model_128.loss_function(real_mask.to(DEVICE1), recon_mask, mu, var)
        loss.backward()
        
        optimizer_mn_128.step()
        
        if i % 20 == 0:
            print('[{}/{}][{}/{}]\t Loss: {}'.format(epoch, num_epochs, i, len(dataloader), loss.item()))
    
    
    recon_mask_wt = wt(recon_mask.to(DEVICE0), filters, levels=2)
    recon_img_wt = Y_32_padded + recon_mask_wt
    recon_img = iwt(recon_img_wt, inv_filters, levels=2)
    
    save_image(real_mask.cpu(), './iwt_model_128_outputs/' + 'real_mask{}.png'.format(epoch))
    save_image(recon_mask.cpu(), './iwt_model_128_outputs/' + 'recon_mask{}.png'.format(epoch))
    save_image(recon_img.cpu(), './iwt_model_128_outputs/' + 'recon_img{}.png'.format(epoch))
    save_image(Y_32_low.cpu(), './iwt_model_128_outputs/' + 'low_img{}.png'.format(epoch))
    
    if epoch % 50 == 0:
        torch.save({
            'model_state_dict': iwt_model_128.state_dict(),
            'optimizer_state_dict': optimizer_mn_128.state_dict(),
            }, 'iwt_model_128_outputs/iwt_model_128_epoch{}.pth'.format(epoch))

In [ ]:
# Load saved 128 and 512 mask networks
checkpoint_128_path = './iwt_model_128_outputs/iwt_model_128.pth'

iwt_model_128 = IWTAE_128_Mask_2(z_dim=100, num_iwt=2)
iwt_model_128.set_filters(inv_filters)
iwt_model_128.set_device(DEVICE1)
iwt_model_128 = iwt_model_128.to(DEVICE0)

checkpoint_128 = torch.load(checkpoint_128_path, map_location=DEVICE0)
iwt_model_128.load_state_dict(checkpoint_128['model_state_dict'])

In [ ]:
iwt_model_128.eval()
iwt_model_512.train()

# Finetuning 128 mask network
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, 0):
        optimizer_mn_512.zero_grad()
        
        real_cpu = data[0].to(DEVICE0)
        Y = wt(real_cpu, filters, levels=4)
        Y_128 = wt(real_cpu, filters, levels=2)[:, :, :128, :128]
        
        # Get real mask at 128 x 128 (2 WT on that so each patch is 32 x 32)
        real_mask = zero_mask(Y, 2, 1)
        real_mask = iwt(real_mask, inv_filters, levels=2)
        
        with torch.no_grad():
            # Get reconstruction from AE & DSVAE
            Y_128_norm = normalize(Y_128, shift, scale)
            Y_128_norm_hat, _, _ = netAE(Y_128_norm)
            Y_128_norm_hat = dsvae(Y_128_norm_hat)
            Y_128_hat = denormalize(Y_128_norm_hat, shift, scale)

            # Apply WT twice to get 32 x 32 low freq patch and pad with zeros to create 128 x 128
            Y_32 = wt(Y_128_hat, filters, levels=2)[:, :, :32, :32]
            Y_32_padded = zero_pad(Y_32, 128, device=DEVICE0)
            Y_32_low = iwt(Y_32_padded, inv_filters, levels=2)

            # Run through 128 mask network and get reconstructed image
            recon_mask_128, _, _ = iwt_model_128(Y_32_low)
            recon_mask_128_wt = wt(recon_mask_128.to(DEVICE0), filters, levels=2)
            recon_mask_128_wt[:, :, :32, :32] = Y_32
            recon_img_128 = iwt(recon_mask_128_wt, inv_filters, levels=2)
        
            # Chain output of first mask network to next
            recon_img_128_padded = zero_pad(recon_img_128, 512, device=DEVICE0)
            recon_img_128_low = iwt(recon_img_128_padded, inv_filters, levels=2)
        
        recon_mask_512, mu, var = iwt_model_512(recon_img_128_low.to(DEVICE1))
        loss, _, _ = iwt_model_512.loss_function(real_mask.to(DEVICE1), recon_mask_512, mu, var)
        loss.backward()
        
        optimizer_mn_512.step()
        
        if i % 50 == 0:
            print('[{}/{}][{}/{}]\t Loss: {}'.format(epoch, num_epochs, i, len(dataloader), loss.item()))
        
        gc.collect()
        torch.cuda.empty_cache()
        
    
    
    recon_mask_512_wt = wt(recon_mask_512.to(DEVICE0), filters, levels=2)
    recon_img_512_wt = recon_img_128_padded + recon_mask_512_wt
    recon_img_512 = iwt(recon_img_512_wt, inv_filters, levels=2)
    
    save_image(real_mask.cpu(), './iwt_model_512_outputs/' + 'real_mask{}.png'.format(epoch))
    save_image(recon_mask_512.cpu(), './iwt_model_512_outputs/' + 'recon_mask{}.png'.format(epoch))
    save_image(recon_img_512.cpu(), './iwt_model_512_outputs/' + 'recon_img{}.png'.format(epoch))
    save_image(recon_img_128_low.cpu(), './iwt_model_512_outputs/' + 'low_img{}.png'.format(epoch))
    
    if epoch % 30 == 0:
        torch.save({
            'model_state_dict': iwt_model_512.state_dict(),
            'optimizer_state_dict': optimizer_mn_512.state_dict(),
            }, 'iwt_model_512_outputs/iwt_model_512_epoch{}.pth'.format(epoch))

In [ ]:
import gc
gc.collect()